In [1]:
import pandas as pd
import numpy as np
import os, sys, math
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import scipy.optimize as opt
from PIL import Image
import cv2
import random as rn

In [2]:
import tensorflow as tf
from tensorflow.image import resize_images

In [3]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model, Model
from keras import metrics
from keras.optimizers import Adam 
from keras import backend as K
from keras import layers
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, LeakyReLU, PReLU, Input
from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, History, TensorBoard, ReduceLROnPlateau
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.densenet import DenseNet201, DenseNet169

Using TensorFlow backend.


In [4]:
os.environ['PYTHONHASHSEED'] = '0'
# Setting the seed for numpy-generated random numbers
np.random.seed(37)
# Setting the seed for python random numbers
rn.seed(1254)
# Setting the graph-level random seed.
tf.set_random_seed(89)
# 自動增長 GPU 記憶體用量
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
# 設定 Keras 使用的 Session
tf.keras.backend.set_session(sess)

In [5]:
name_label_dict = {
0:  'Nucleoplasm',
1:  'Nuclear membrane',
2:  'Nucleoli',   
3:  'Nucleoli fibrillar center',
4:  'Nuclear speckles',
5:  'Nuclear bodies',
6:  'Endoplasmic reticulum',   
7:  'Golgi apparatus',
8:  'Peroxisomes',
9:  'Endosomes',
10:  'Lysosomes',
11:  'Intermediate filaments',
12:  'Actin filaments',
13:  'Focal adhesion sites',   
14:  'Microtubules',
15:  'Microtubule ends',  
16:  'Cytokinetic bridge',   
17:  'Mitotic spindle',
18:  'Microtubule organizing center',  
19:  'Centrosome',
20:  'Lipid droplets',
21:  'Plasma membrane',   
22:  'Cell junctions', 
23:  'Mitochondria',
24:  'Aggresome',
25:  'Cytosol',
26:  'Cytoplasmic bodies',   
27:  'Rods & rings' }

In [6]:
NAME = "test5_notgenerator_DenseNet169_dense"# os.path.basename(__file__).split('.')[0]
PATH = os.getcwd()
TRAIN = os.path.join(os.getcwd(), 'data', 'train')
TEST = os.path.join(os.getcwd(), 'data', 'test')
PREPROCESSED = os.path.join(os.getcwd(), 'preprocessed_data')
LABELS = os.path.join(os.getcwd(), 'data', 'train.csv')
SAMPLE = os.path.join(os.getcwd(), 'data', 'sample_submission.csv')
MODEL = os.path.join(os.getcwd(), 'model', NAME+'.h5')
RESULT = os.path.join(os.getcwd(), 'result', NAME+'_submission.csv')

In [7]:
IMAGE_LENGTH = 512
IMAGE_WIDTH = 512
CHANNEL_NUM = 4
TRAIN_SIZE = int(len(os.listdir(TRAIN))/4)
LABEL_NUM = 28

In [8]:
train_x = np.load(os.path.join(PREPROCESSED, 'train_RGBY_original_x.npy'))
train_y = np.load(os.path.join(PREPROCESSED, 'train_RGBY_original_y.npy'))


In [9]:
# Use ImageDataGenerator to implement data augmentation. 
datagen = ImageDataGenerator(
            rotation_range = 40,
            width_shift_range = 0.3,
            height_shift_range = 0.3,
            zoom_range = [0.6, 1.4],
            shear_range = 0.4,
            horizontal_flip = True)

In [10]:
def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

In [11]:
def create_model(input_shape, n_out):
    
    input_tensor = Input(shape=input_shape)
    bn = BatchNormalization()(input_tensor)
    x = Dense(CHANNEL_NUM)(bn)
    x = Dense(3)(x)
    #conv2d = Conv2D(3, kernel_size = (1, 1), strides=(1,1), padding = 'same', kernel_initializer = 'glorot_normal')(bn)
    
    #x = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(IMAGE_LENGTH, IMAGE_WIDTH, 3), pooling='avg')(conv2d)
    x = DenseNet169(include_top=False, weights='imagenet', input_shape=(IMAGE_LENGTH, IMAGE_WIDTH, 3), pooling='avg')(x)
    
    #x = Conv2D(128, kernel_size=(1,1), activation='relu')(x)
    #x = Flatten()(x)
    #x = Dropout(0.5)(x)
    #x = Dense(512, activation='relu')(x)
    #x = Dropout(0.5)(x)
    output = Dense(n_out, activation='softmax')(x)
    model = Model(input_tensor, output)
    
    return model

In [12]:
model = create_model(
    input_shape=(IMAGE_LENGTH,IMAGE_WIDTH,CHANNEL_NUM), 
    n_out=LABEL_NUM)

In [13]:
for layer in model.layers:
    layer.trainable = True
model.layers[-2].trainable = False
### Show model summary
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 512, 512, 4)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512, 512, 4)       16        
_________________________________________________________________
dense_1 (Dense)              (None, 512, 512, 4)       20        
_________________________________________________________________
dense_2 (Dense)              (None, 512, 512, 3)       15        
_________________________________________________________________
densenet169 (Model)          (None, 1664)              12642880  
_________________________________________________________________
dense_3 (Dense)              (None, 28)                46620     
Total params: 12,689,551
Trainable params: 46,663
Non-trainable params: 12,642,888
___________________________________________________________

In [14]:
adam = Adam(lr=1e-4)
epochs_to_wait_for_improve = 10
batch_size = 4
#valid_split_ratio = 0.2
n_epochs = 4

In [15]:
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    # tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [16]:
cw = np.load(os.path.join(PREPROCESSED, 'class_weight.npy'))

In [17]:
model.compile(loss=focal_loss(), optimizer=adam, metrics=[f1]) 

In [18]:
#early_stopping_callback = EarlyStopping(monitor='val_f1', patience=epochs_to_wait_for_improve)

checkpoint_callback = ModelCheckpoint(MODEL, monitor='val_f1'
                                          , verbose=1, save_best_only=True, mode='max')

In [19]:

train_history=model.fit(train_x, train_y
                                      , batch_size = batch_size
                                      , epochs = n_epochs
                                      , validation_split = 0.1
                                    , shuffle=True, class_weight=cw
                                      , callbacks=[ 
                                          checkpoint_callback, TensorBoard(log_dir='./tmp/log')
                                                   , ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001)
                                                  ]
                                      , verbose=1)
                    

C:\Users\OneDay\Anaconda3\envs\ML2018FALL_NEW\lib\site-packages\keras\callbacks.py:928: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` insted.
  warnings.warn('`epsilon` argument is deprecated and '


Train on 27964 samples, validate on 3108 samples
Epoch 1/4
27964/27964 [==============================] - 1516s 54ms/step - loss: 3.3340 - f1: 2.5713e-04 - val_loss: 4.0083 - val_f1: 0.0024

Epoch 00001: val_f1 improved from -inf to 0.00240, saving model to C:\Users\OneDay\Downloads\ML2018FALL\final\Human_Protein_Atlas_Image_classification\model\test5_notgenerator_DenseNet169_dense.h5
Epoch 2/4
27964/27964 [==============================] - 1488s 53ms/step - loss: 3.0173 - f1: 0.0016 - val_loss: 4.3045 - val_f1: 0.0056

Epoch 00002: val_f1 improved from 0.00240 to 0.00555, saving model to C:\Users\OneDay\Downloads\ML2018FALL\final\Human_Protein_Atlas_Image_classification\model\test5_notgenerator_DenseNet169_dense.h5
Epoch 3/4
27964/27964 [==============================] - 1486s 53ms/step - loss: 2.8836 - f1: 0.0030 - val_loss: 4.2536 - val_f1: 0.0057

Epoch 00003: val_f1 improved from 0.00555 to 0.00568, saving model to C:\Users\OneDay\Downloads\ML2018FALL\final\Human_Protein_Atlas_Ima

In [20]:
#!tensorboard --logdir=./tmp/log

In [21]:
for layer in model.layers:
        layer.trainable = True
model.layers[-2].trainable = True
### Show model summary
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 512, 512, 4)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512, 512, 4)       16        
_________________________________________________________________
dense_1 (Dense)              (None, 512, 512, 4)       20        
_________________________________________________________________
dense_2 (Dense)              (None, 512, 512, 3)       15        
_________________________________________________________________
densenet169 (Model)          (None, 1664)              12642880  
_________________________________________________________________
dense_3 (Dense)              (None, 28)                46620     
Total params: 205,071
Trainable params: 46,663
Non-trainable params: 158,408
_________________________________________________________________

C:\Users\OneDay\Anaconda3\envs\ML2018FALL_NEW\lib\site-packages\keras\engine\training.py:975: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [22]:
n_epochs = 20

In [23]:
model.compile(loss=focal_loss(), optimizer=adam, metrics=[f1]) 

In [24]:
train_history=model.fit(train_x, train_y
                                      , batch_size = batch_size
                                      #, steps_per_epoch = len(train_x)*10 / batch_size
                                      , epochs = n_epochs
                                      , validation_split = 0.1
                                    , shuffle=True, class_weight=cw
                                      , callbacks=[
                                          #early_stopping_callback, 
                                          checkpoint_callback, TensorBoard(log_dir='./tmp/log')
                                                   , ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.00001)
                                                  ]
                                      , verbose=1)

Train on 27964 samples, validate on 3108 samples
Epoch 1/20
27964/27964 [==============================] - 2234s 80ms/step - loss: 2.4006 - f1: 0.0182 - val_loss: 2.3571 - val_f1: 0.0437

Epoch 00001: val_f1 improved from 0.00631 to 0.04374, saving model to C:\Users\OneDay\Downloads\ML2018FALL\final\Human_Protein_Atlas_Image_classification\model\test5_notgenerator_DenseNet169_dense.h5
Epoch 2/20
27964/27964 [==============================] - 2197s 79ms/step - loss: 1.9579 - f1: 0.0312 - val_loss: 2.2104 - val_f1: 0.0577

Epoch 00002: val_f1 improved from 0.04374 to 0.05768, saving model to C:\Users\OneDay\Downloads\ML2018FALL\final\Human_Protein_Atlas_Image_classification\model\test5_notgenerator_DenseNet169_dense.h5
Epoch 3/20
27964/27964 [==============================] - 2204s 79ms/step - loss: 1.7114 - f1: 0.0409 - val_loss: 1.8473 - val_f1: 0.0559

Epoch 00003: val_f1 did not improve from 0.05768
Epoch 4/20
27964/27964 [==============================] - 2220s 79ms/step - loss: 1.5

In [25]:
train_history=model.fit(train_x, train_y
                                      , batch_size = batch_size
                                      #, steps_per_epoch = len(train_x)*10 / batch_size
                                      , epochs = n_epochs
                                      , validation_split = 0.1
                                    , shuffle=True, class_weight=cw
                                      , callbacks=[
                                          #early_stopping_callback, 
                                          checkpoint_callback, TensorBoard(log_dir='./tmp/log')
                                                   , ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.00001)
                                                  ]
                                      , verbose=1)

Train on 27964 samples, validate on 3108 samples
Epoch 1/20
27964/27964 [==============================] - 2197s 79ms/step - loss: 0.5610 - f1: 0.0955 - val_loss: 1.6045 - val_f1: 0.0877

Epoch 00001: val_f1 did not improve from 0.08784
Epoch 2/20
27964/27964 [==============================] - 2198s 79ms/step - loss: 0.5647 - f1: 0.0954 - val_loss: 1.5987 - val_f1: 0.0870

Epoch 00002: val_f1 did not improve from 0.08784
Epoch 3/20
27964/27964 [==============================] - 2204s 79ms/step - loss: 0.5626 - f1: 0.0949 - val_loss: 1.5960 - val_f1: 0.0876

Epoch 00003: val_f1 did not improve from 0.08784
Epoch 4/20
27964/27964 [==============================] - 2211s 79ms/step - loss: 0.5587 - f1: 0.0954 - val_loss: 1.6345 - val_f1: 0.0876

Epoch 00004: val_f1 did not improve from 0.08784
Epoch 5/20
27964/27964 [==============================] - 2197s 79ms/step - loss: 0.5531 - f1: 0.0957 - val_loss: 1.6233 - val_f1: 0.0868

Epoch 00005: val_f1 did not improve from 0.08784

Epoch 0000